In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout
from keras.layers import Flatten
from keras.layers import Dense

C:\Users\Appzone-MLG\Anaconda3\envs\chest_xray_env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# initializing the classfier
classifier = Sequential()

# Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (350, 350, 1), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.2))

classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.2))

classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.2))

classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.2))

# Flattening
classifier.add(Flatten())

# Fully connected layer
classifier.add(Dense(activation = 'relu', units = 128))
classifier.add(Dropout(0.3))
classifier.add(Dense(activation = 'relu', units = 64))
classifier.add(Dropout(0.3))
classifier.add(Dense(activation = 'relu', units = 64))
classifier.add(Dropout(0.3))
classifier.add(Dense(activation = 'sigmoid', units = 14))

classifier.compile(optimizer = 'adam',
                   loss ='binary_crossentropy',
                   metrics = ['accuracy']
                  )







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 348, 348, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 174, 174, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 174, 174, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 172, 172, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 86, 86, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 86, 86, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 84, 84, 64)        36928     
__________

In [5]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

In [6]:
# to display all the columns of the dataframe in the notebook
pd.pandas.set_option('display.max_columns', None)

In [7]:
# Reading the csv file 
# Here we will be using a subset of the original dataset

train = pd.read_csv('CheXpert-v1.0-small/train_data.csv', nrows=1000)

In [8]:
train.head()

,path,no_finding,enlarged_cardiomediastinum,cardiomegaly,lung_opacity,lung_lesion,edema,consolidation,pneumonia,atelectasis,pneumothorax,pleural_effusion,pleural_other,fracture,support_devices,no_of_labels
0,CheXpert-v1.0-small/train/patient00001/study1/...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
2,CheXpert-v1.0-small/train/patient00002/study1/...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
3,CheXpert-v1.0-small/train/patient00002/study1/...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
4,CheXpert-v1.0-small/train/patient00003/study1/...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [65]:
# loading and preprocessing 
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img(''+train['path'][i]+'', target_size=(350, 350, 1), color_mode = "grayscale")
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 463.55it/s]


In [66]:
# X.shape

In [67]:
# X[0]

In [69]:
# plt.imshow(X[0], cmap='gray')
# plt.imshow(X[2], cmap='gray')

In [34]:
# Extracting the Onehot encoded label
y = np.array(train.drop(['path', 'no_of_labels'],axis=1))
y.shape

(1000, 14)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1)

In [72]:
classifier.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)

Train on 900 samples, validate on 100 samples
Epoch 1/50
900/900 [==============================] - 6s 7ms/step - loss: 0.3330 - acc: 0.8637 - val_loss: 0.3689 - val_acc: 0.8407: 1s - loss: 0.3328 - acc: 0.86 - ETA: 0s - loss: 0.3330 - acc: 0.
Epoch 2/50
900/900 [==============================] - 5s 6ms/step - loss: 0.3272 - acc: 0.8646 - val_loss: 0.3682 - val_acc: 0.8379
Epoch 3/50
900/900 [==============================] - 6s 6ms/step - loss: 0.3186 - acc: 0.8688 - val_loss: 0.3677 - val_acc: 0.8379
Epoch 4/50
900/900 [==============================] - 6s 6ms/step - loss: 0.3153 - acc: 0.8693 - val_loss: 0.3699 - val_acc: 0.8329
Epoch 5/50
900/900 [==============================] - 5s 6ms/step - loss: 0.3046 - acc: 0.8740 - val_loss: 0.3656 - val_acc: 0.8329
Epoch 6/50
900/900 [==============================] - 5s 6ms/step - loss: 0.2999 - acc: 0.8765 - val_loss: 0.3707 - val_acc: 0.8421
Epoch 7/50
900/900 [==============================] - 5s 6ms/step - loss: 0.2965 - acc: 0.8763 -

In [73]:
# Using image generators
from keras.preprocessing.image import ImageDataGenerator

In [87]:
train_df = pd.read_csv("CheXpert-v1.0-small/train_data.csv", nrows=1600)
valid_df = pd.read_csv("CheXpert-v1.0-small/valid.csv")

train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        horizontal_flip=False)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=None,
        x_col="path",
        y_col=['no_finding',
               'enlarged_cardiomediastinum',
               'cardiomegaly',
               'lung_opacity',
               'lung_lesion',
               'edema',
               'consolidation',
               'pneumonia',
               'atelectasis',
               'pneumothorax',
               'pleural_effusion',
               'pleural_other',
               'fracture',
               'support_devices'
               ],
        color_mode="grayscale",
        target_size=(350, 350),
        batch_size=32,
#         class_mode="multi_output",
        class_mode="raw",
)

validation_generator = test_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=None,
        x_col="Path",
        y_col=['No Finding',
               'Enlarged Cardiomediastinum',
               'Cardiomegaly',
               'Lung Opacity',
               'Lung Lesion',
               'Edema',
               'Consolidation',
               'Pneumonia',
               'Atelectasis',
               'Pneumothorax',
               'Pleural Effusion',
               'Pleural Other',
               'Fracture',
               'Support Devices'
              ],
        color_mode="grayscale",
        target_size=(350, 350),
        batch_size=128,
        class_mode="raw")

classifier.fit_generator(
        train_generator,
        steps_per_epoch=50,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=2)

# Validation has 235 images

# model.fit_generator(train_generator,
#                     #epochs = 3, we will use epoch after I implement data arguementation
#                     steps_per_epoch = 75,
#                     validation_data = validation_generator,
#                     validation_steps = 2
#                    )

Found 1600 validated image filenames.
Found 234 validated image filenames.
Epoch 1/50
50/50 [==============================] - 21s 428ms/step - loss: 0.3518 - acc: 0.8546 - val_loss: 0.4492 - val_acc: 0.8016
Epoch 2/50
50/50 [==============================] - 19s 372ms/step - loss: 0.3448 - acc: 0.8550 - val_loss: 0.4485 - val_acc: 0.7943
Epoch 3/50
50/50 [==============================] - 20s 391ms/step - loss: 0.3400 - acc: 0.8590 - val_loss: 0.4495 - val_acc: 0.7988
Epoch 4/50
50/50 [==============================] - 21s 419ms/step - loss: 0.3367 - acc: 0.8575 - val_loss: 0.4495 - val_acc: 0.7988
Epoch 5/50
50/50 [==============================] - 19s 378ms/step - loss: 0.3331 - acc: 0.8621 - val_loss: 0.4494 - val_acc: 0.7955
Epoch 6/50
50/50 [==============================] - 20s 405ms/step - loss: 0.3315 - acc: 0.8616 - val_loss: 0.4436 - val_acc: 0.7949
Epoch 7/50
50/50 [==============================] - 23s 460ms/step - loss: 0.3258 - acc: 0.8655 - val_loss: 0.4668 - val_acc: 0

In [89]:
# Single preds

from keras.preprocessing import image 

In [107]:
test_image = image.load_img('CheXpert-v1.0-small/valid/patient64740/study1/view1_frontal.jpg',
                            target_size = (350, 350),
                            color_mode= 'grayscale'
                           )

In [108]:
print(test_image)

<PIL.Image.Image image mode=L size=350x350 at 0x17202CF9CF8>


In [109]:
test_image_arr = image.img_to_array(test_image)

In [110]:
# test_image_arr.shape

In [111]:
test_image_arr = np.expand_dims(test_image_arr, axis = 0)

In [112]:
classifier.predict(test_image_arr)

array([[0.9999999, 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       ]], dtype=float32)